Running a simple simulation and exporting the battery:

In [1]:
from PySAM.BatteryStateful import default
from PySAM.BatteryTools import battery_model_sizing

battery = default("LFPGraphite")

battery_model_sizing(
    model=battery,
    desired_power=5.1,
    desired_capacity=5.12,
    desired_voltage=204,
)
battery.ParamsCell.minimum_SOC = 5
battery.ParamsCell.maximum_SOC = 95
battery.ParamsCell.initial_SOC = 50
battery.Controls.replace({
    "control_mode": 1,
    "dt_hr": 1,
    "input_power": 0,
})
battery.setup()
for i in [-1, 2, 1, 0.5, 0, -2, -1.5, -0.5, 1]:
    battery.Controls.input_power = i
    battery.execute(0)

exported_state = battery.export()
del exported_state["Controls"]

Creating a new battery out of the exported state, then comparing the new exported state

In [2]:
from deepdiff import DeepDiff
from PySAM.BatteryStateful import new

battery = new()
battery.ParamsCell.assign(exported_state["ParamsCell"])
battery.ParamsPack.assign(exported_state["ParamsPack"])
battery.Controls.replace({
    "control_mode": 1,
    "dt_hr": 1,
    "input_power": 0,
})
battery.setup()
battery.StateCell.assign(exported_state["StateCell"])
battery.StatePack.assign(exported_state["StatePack"])
new_exported_state = battery.export()
del new_exported_state["Controls"]

DeepDiff(exported_state, new_exported_state)

{}